In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, col, broadcast, collect_set, array_contains, size

In [ ]:


class Transformer:
    """
    This is where all the business logics are built and tranformed.

    """
    def __init__(self):
        pass

    def transform(self, input_df):
        pass

class LeadTransformer(Transformer):

    def transform(self, input_df):
        
        transcation_input_df = input_df.get("transaction_input_df")

        window_spec = Window.partitionBy("customer_id").orderBy("transaction_date")

        transformed_df = transcation_input_df.withColumn(
            "next_product_name", lead("product_name").over(window_spec)
        )

        # print(transformed_df.columns)
        # Airpods bought after Iphone
        transformed_df.orderBy("customer_id","transaction_date","product_name").show()

        
        # Customer who actually bought Airpds after buying Iphone.
        filtered_df = transformed_df.filter(
            (col("product_name") == "iPhone") & (col("next_product_name") == "AirPods")
        )
        filtered_df.orderBy("customer_id", "transaction_date", "product_name").show()

        customer_input_df = input_df.get("customer_input_df")
        # join_trans_filtered_df = 
        join_df = filtered_df.join(
            broadcast(customer_input_df),
            "customer_id",
        )

        # join_df.show()
        return join_df.select(
            "customer_id",
            "customer_name",
            "location"
            )

class OnlyAirpodsAndIphone(Transformer):
    def transform(self, input_dfs):
        """
        Customer who have bought only Iphone and Airpods nothing else.
        """
        transaction_input_df = input_dfs.get("transaction_input_df")
        grouped_df = transaction_input_df.groupBy("customer_id").agg(
            collect_set("product_name").alias("products")
        )

        print("Grouped DF")
        grouped_df.show()

        filtered_df = grouped_df.filter(
            (array_contains(col("products"), "iphone") ) &
            (array_contains(col("products"), "AirPods")) &
            (size(col("products")) == 2)
        )
        print("Only Airpods and iPhone")
        filtered_df.show()
        customer_input_df = input_dfs.get("customerInputDF")

        customer_input_df.show()

        join_df =  customer_input_df.join(
           broadcast(filtered_df),
            "customer_id"
        )

        print("JOINED DF")
        join_df.show()

        return join_df.select(
            "customer_id",
            "customer_name",
            "location"
        )
        